In [ ]:
import pickle as pkl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from src.plotting_tools.Bins import bins
import ROOT as r
import array

In [ ]:
def make_hist(values, errors, *args):
    hpx    = r.TH1F(*args, bins.calc_nBins(), array.array('d', bins.bin_edges))
    for i, (x,e) in enumerate(zip(values,errors)):
        hpx.SetBinContent(i, x) 
        hpx.SetBinError(i, e) 
    return hpx


In [ ]:
# http://cms-analysis.github.io/HiggsAnalysis-CombinedLimit/part2/settinguptheanalysis/
# https://github.com/cms-analysis/HiggsAnalysis-CombinedLimit/blob/102x/data/tutorials/shapes/simple-shapes-df_input.csv

In [ ]:
mass_hist_template = 'sig{}'

In [ ]:
era = '2016'
with open('data/{}_bff_interp.pkl'.format(era), 'rb') as f:
    data = pkl.load(f)

In [ ]:
outname="fits/limit_setting/abcd_dict_{}.pkl".format(era)
with open(outname,'rb') as f:
    abcd = pkl.load(f)

In [ ]:
def rebin(oldbins, newbins, data, drawplot=False):
    fig, ax = plt.subplots()
    values = ax.hist(oldbins, newbins, weights = data)
    if not drawplot: plt.close(fig)
    return values[0]

In [ ]:
def rebin_np(oldbins, newbins, data, drawplot=False):
    values = np.histogram(oldbins, newbins, weights = data)
    return values[0]

In [ ]:
from time import perf_counter

In [ ]:

sys_map= {
"{reg}_nom": "nominal", 
"{reg}_jet_jerUp_muon_corrected_pt_ele_pt_{sys_dir}": "jer_{era}_{sys_dir}",
"{reg}_jet_jesTotalUp_muon_corrected_pt_ele_pt_{sys_dir}": "jes_{era}_{sys_dir}",
"{reg}_jet_nom_muon_correctedUp_pt_ele_pt_{sys_dir}": "roch_{era}_{sys_dir}",
"{reg}_jet_jesHEMIssueUp_muon_corrected_pt_ele_pt_{sys_dir}": "HEM_{era}_{sys_dir}",
"{reg}_Weight_BTagUp_{sys_dir}": "btag_{era}_{sys_dir}",
"{reg}_Weight_ElectronSFUp_{sys_dir}": "elSF_{era}_{sys_dir}",
"{reg}_Weight_ISRFSR_Up_{sys_dir}": "ISRFSR_{era}_{sys_dir}",
"{reg}_Weight_L1Up_{sys_dir}": "L1_{era}_{sys_dir}",
"{reg}_Weight_MuonSFUp_{sys_dir}": "Muon_{era}_{sys_dir}",
"{reg}_Weight_MuonTriggerUp_{sys_dir}": "trigger_{era}_{sys_dir}",
"{reg}_Weight_PDF_Up_{sys_dir}": "pdf_{era}_{sys_dir}",
"{reg}_Weight_PUIDUp_{sys_dir}": "puid_{era}_{sys_dir}",
"{reg}_Weight_PuUp_{sys_dir}": "pu_{sys_dir}"}


#no era in PU as correlated

In [ ]:
import re
def get_dbs(string):
    dbs = re.findall('([0-9].[0-9]{2})', string)
    if len(dbs)==1: return float(dbs[0])
    return 0

In [ ]:
dbses  = [0 , 0.04, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]
masses = np.linspace(125,400,int((400-125)/7+1), dtype=int)
#dbses = [0 , 0.04]
#masses = masses[0:1]
masses, dbses

In [ ]:
#hfile = r.TFile('./exo-datacards/EXO-22-006/{era}/{era}_background.root'.format(era=era), 'RECREATE')
#c1 = r.TCanvas( 'c1', 'Dynamic Filling Example', 200, 10, 700, 500 )

In [ ]:
hfile = r.TFile('./combine_data/{era}/{era}_shapes_df_input.root'.format(era=era), 'RECREATE')
c1 = r.TCanvas( 'c1', 'combine canvas', 200, 10, 700, 500 )

In [ ]:
# abcd prediction
#bin_list = []
for reg in abcd:
    print(reg)
    noms = abcd[reg]['nom']
    stds = abcd[reg]['std']
    #for i, (nom, std) in enumerate(zip(noms, stds)):
    #    bin_list.append({"channel": reg, 
    #                     "systematic": "sys_0_nominal",
    #                     "bin": i, "sum_w": nom, "sum_ww": std**2,
    #                    "process": 'background'
    #                    })        
    #    bin_list.append({"channel": reg, 
    #                     "systematic": "sys_0_nominal",
    #                     "bin": i, "sum_w": nom, "sum_ww": std**2,
    #                    "process": 'data_obs'
    #                    })  
    hist_name = '{}-{}-{}'.format(reg, "sys_0_nominal", 'data_obs')
    hist = make_hist(noms, stds, hist_name, hist_name)
    hist.Write()

    print(hist_name)
    hist_name = '{}-{}-{}'.format(reg, "sys_0_nominal", 'background')
    hist =  make_hist(noms, stds, hist_name, hist_name)
    hist.Write()

#abcd_bin_list = bin_list
#df = pd.DataFrame(bin_list)
#df['channel'] = df.channel.astype('str')
#df['systematic'] = df.systematic.astype('str')
#df['process'] = df.process.astype('str')
#exodc_path = './exo-datacards/EXO-22-006/2017/{}_abcd_df_input.csv'.format(era)
#df[['channel', 'process', 'systematic', 'bin', 'sum_w','sum_ww']].to_csv(exodc_path, index=False)

In [ ]:
#hfile.Write()
#hfile.Close()

In [ ]:
#hfile = r.TFile('./exo-datacards/EXO-22-006/{era}/{era}_shapes_df_input.root'.format(era=era), 'RECREATE')
#c1 = r.TCanvas( 'c1', 'Dynamic Filling Example', 200, 10, 700, 500 )

In [ ]:
for mass in masses:
    #bin_list = []
    for reg in ['SR1', 'SR2']:
        print(mass)
        nom = rebin_np(data[mass]['bins'].bin_edges, bins.bin_edges,
                    data[mass]['{}_nom_0.00'.format(reg)])
        for key in data[mass]:
            for dbs in dbses:
                if key == 'bins': continue
                if reg not in key: continue
                sys_dir = 'down' if 'down' in key else 'up'
                sys_dbs = get_dbs(key)
                if get_dbs(key) != dbs: continue
                if key == 'bins': continue
                if reg not in key: continue
                    
                systematic = 'NA'
                for sys in sys_map:
                    if sys.format(reg=reg, era=era, sys_dir=sys_dir)+"_{:.2f}".format(dbs)==key:
                        systematic = sys_map[sys].format(reg=reg, era=era, sys_dir=sys_dir.capitalize())
                        break
                if systematic=='NA': 
                    print("----")
                    print(key)
                    for sys in sys_map:
                        print("\t",sys.format(reg=reg, era=era, sys_dir=sys_dir)+"_{:.2f}".format(dbs))
                values = rebin_np(data[mass]['bins'].bin_edges, bins.bin_edges, data[mass][key])
                if 'Trigger' in key:
                    if ('up' in key) and ('0.4' in key): 
                        print(key, values.sum())
#                    if 'up' in key:
#                        values = values + nom
#                    else:
#                        values = -values + nom
#                #print(key, np.sum(values))
#                #for i, value in enumerate(values):
#                #    if 'Trigger' in key:
#                #        value = value +1
#                #    bin_list.append({"channel": reg, 
#                #                     "systematic": "sys_{}_{}".format(dbs, systematic),
#                #                     "bin": i, "sum_w": value, "sum_ww": value,
#                #                    "process": int(mass)
#                #                    })
#                if 'Trigger' in key:
#                        values = values +1
#                mass_proc = mass_hist_template.format(int(mass))
#                hist_name = '{}-{}-{}'.format(reg, "sys_{}_{}".format(dbs, systematic),  mass_proc)
#                hist = make_hist(values, values, hist_name, hist_name)
#                hist.Write()
#            
#        #bin_list += abcd_bin_list
#        #make df
#        #df = pd.DataFrame(bin_list)
#        #df['channel'] = df.channel.astype('str')
#        #df['systematic'] = df.systematic.astype('str')
#        #df['process'] = df.process.astype('str')        
#        #exodc_path = '/afs/cern.ch/work/r/rymuelle/public/nanoAODzPrime/CMSSW_12_1_0/src/bff_plotting_code_v3/exo-datacards/EXO-22-006/2017/{}_{}_shapes_df_input.csv'.format(era, mass)
#        #df[['channel', 'process', 'systematic', 'bin', 'sum_w','sum_ww']].to_csv(exodc_path, index=False)
#            

In [ ]:
if era=="2018":
    template = '''#higgs combine tool shape analysis card for z'to mumu 1 jet
#https://github.com/cms-analysis/HiggsAnalysis-CombinedLimit/blob/102x/data/tutorials/shapes/simple-shapes-df.txt
-------------------------

imax 1  number of channels                                      #1 Jet
jmax 1  number of backgrounds -1                                    #following AN2015_207_v5, not sure why the -1 is there?
kmax *  number of nuisance parameters (sources of systematic uncertainties)

-------------------------


shapes * * {era}_shapes_df_input.root $CHANNEL-sys_0_nominal-$PROCESS $CHANNEL-$SYSTEMATIC-$PROCESS

shapes background * {era}_shapes_df_input.root $CHANNEL-sys_0_nominal-background

-------------------------

bin       {reg}
observation   -1

-------------------------

bin       {reg}       {reg}
process     background    {mass_proc}
process     1     -1
rate      -1   -1

-------------------------
lumi lnN -      {lumi}
sys_{dbs}_jer_{era}_   shapeN2 -      1
sys_{dbs}_jes_{era}_   shapeN2 -      1
sys_{dbs}_roch_{era}_   shapeN2 -      1
sys_{dbs}_HEM_{era}_   shapeN2 -      1
sys_{dbs}_btag_{era}_   shapeN2 -      1
sys_{dbs}_elSF_{era}_   shapeN2 -      1
sys_{dbs}_ISRFSR_{era}_   shapeN2 -      1
sys_{dbs}_Muon_{era}_   shapeN2 -      1
sys_{dbs}_trigger_{era}_   shapeN2 -      1
sys_{dbs}_pdf_{era}_   shapeN2 -      1
sys_{dbs}_puid_{era}_   shapeN2 -      1
sys_{dbs}_pu_   shapeN2 -      1
--------------------------------------------------------------------------------'''
else:
    template = '''#higgs combine tool shape analysis card for z'to mumu 1 jet
#https://github.com/cms-analysis/HiggsAnalysis-CombinedLimit/blob/102x/data/tutorials/shapes/simple-shapes-df.txt
-------------------------

imax 1  number of channels                                      #1 Jet
jmax 1  number of backgrounds -1                                    #following AN2015_207_v5, not sure why the -1 is there?
kmax *  number of nuisance parameters (sources of systematic uncertainties)

-------------------------


shapes * * {era}_shapes_df_input.root $CHANNEL-sys_0_nominal-$PROCESS $CHANNEL-$SYSTEMATIC-$PROCESS

shapes background * {era}_shapes_df_input.root $CHANNEL-sys_0_nominal-background

-------------------------

bin       {reg}
observation   -1

-------------------------

bin       {reg}       {reg}
process     background    {mass_proc}
process     1     -1
rate      -1   -1

-------------------------
lumi lnN -      {lumi}
sys_{dbs}_jer_{era}_   shapeN2 -      1
sys_{dbs}_jes_{era}_   shapeN2 -      1
sys_{dbs}_roch_{era}_   shapeN2 -      1
sys_{dbs}_btag_{era}_   shapeN2 -      1
sys_{dbs}_elSF_{era}_   shapeN2 -      1
sys_{dbs}_ISRFSR_{era}_   shapeN2 -      1
sys_{dbs}_L1_{era}_   shapeN2 -      1
sys_{dbs}_Muon_{era}_   shapeN2 -      1
sys_{dbs}_trigger_{era}_   shapeN2 -      1
sys_{dbs}_pdf_{era}_   shapeN2 -      1
sys_{dbs}_puid_{era}_   shapeN2 -      1
sys_{dbs}_pu_   shapeN2 -      1
--------------------------------------------------------------------------------'''        

In [ ]:
if era=='2016':
    lumi=1.025
if era=='2017':
    lumi=1.023
if era=='2018':
    lumi=1.025

In [ ]:
for reg in ['SR1', 'SR2']:
    for mass in masses:
            for dbs in dbses:
                mass_proc = mass_hist_template.format(int(mass))
                tmp = template.format(era=era,dbs=dbs,mass_proc=mass_proc,lumi=lumi, reg=reg)
                #with open('exo-datacards/EXO-22-006/comb_{}_{}_{}.txt'.format(reg, mass, dbs), 'w') as f:
                with open('combine_data/{}/{}_{}_BFFZprimeToMuMu_fit_M_{}_dbs{}.txt'.format(era, era, reg, mass, 
                                                                                                    str(dbs).replace('.','p')), 'w') as f:
                    f.write(tmp)
                

In [ ]:
hfile.Write()

In [ ]:
template_csv = '''#higgs combine tool shape analysis card for z'to mumu 1 jet
#https://github.com/cms-analysis/HiggsAnalysis-CombinedLimit/blob/102x/data/tutorials/shapes/simple-shapes-df.txt
-------------------------

imax 1  number of channels                                      #1 Jet
jmax 1  number of backgrounds -1                                    #following AN2015_207_v5, not sure why the -1 is there?
kmax *  number of nuisance parameters (sources of systematic uncertainties)

-------------------------


shapes * * {era}_{mass}_shapes_df_input.csv $CHANNEL:$PROCESS:sys_0_nominal,sum_w:sum_ww $CHANNEL:$PROCESS:$SYSTEMATIC,sum_w:sum_ww

-------------------------

bin       SR1
observation   -1

-------------------------

bin       SR1       SR1
process     background    {mass}
process     1     -1
rate      -1   -1

-------------------------
lumi lnN -      {lumi}
sys_{dbs}_jer_{era}_   shapeN2 -      1
sys_{dbs}_jes_{era}_   shapeN2 -      1
sys_{dbs}_roch_{era}_   shapeN2 -      1
sys_{dbs}_btag_{era}_   shapeN2 -      1
sys_{dbs}_elSF_{era}_   shapeN2 -      1
sys_{dbs}_ISRFSR_{era}_   shapeN2 -      1
sys_{dbs}_L1_{era}_   shapeN2 -      1
sys_{dbs}_Muon_{era}_   shapeN2 -      1
sys_{dbs}_trigger_{era}_   shapeN2 -      1
sys_{dbs}_pdf_{era}_   shapeN2 -      1
sys_{dbs}_puid_{era}_   shapeN2 -      1
sys_{dbs}_pu_   shapeN2 -      1
--------------------------------------------------------------------------------'''